### Whn you run this script we are assuming that you already have the following libraries:

- tensorflow: 2.19.0
- torch: 2.7.1+cu128
- transformers: 4.54.0
- sklearn: 1.7.1 
- xgboost: 3.0.2 
- lightgbm: 4.6.0 


In [4]:
import sys, torch, tensorflow as tf, sklearn, transformers, xgboost, importlib
print(sys.executable)
print("torch:", torch.__version__)
print("tf:", tf.__version__)
print("sklearn:", sklearn.__version__)
importlib.import_module("transformers"); importlib.import_module("xgboost");


2025-11-01 20:38:16.541977: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-01 20:38:16.549010: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1762047496.557327  744369 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1762047496.560051  744369 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1762047496.567015  744369 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

/home/falasoul/venvs/ai-env/bin/python
torch: 2.7.1+cu128
tf: 2.19.0
sklearn: 1.6.1


In [14]:
# --- Project Root Setup ---
from pathlib import Path
import sys

# Find project root (one level above notebooks/)
root = Path("..").resolve()

# Add to sys.path so you can import from src/ anywhere
sys.path.append(str(root / "src"))

print(f"✅ Project root set to: {root}")
print(f"📂 Working notebook in: {Path.cwd()}")


✅ Project root set to: /home/falasoul/notebooks/USD/AAI-590/Capstone/AAI-590-G3-ASL
📂 Working notebook in: /home/falasoul/notebooks/USD/AAI-590/Capstone/AAI-590-G3-ASL/notebooks


In [2]:
import sys, subprocess

def pip_install(pkg):
    subprocess.run([sys.executable, "-m", "pip", "install", "--upgrade", pkg], check=True)

# Minimal, non-conflicting additions
for pkg in [
    "yt-dlp",               # video downloads
    "opencv-python",        # lightweight video/image utils
    "decord==0.6.0",        # fast video reader (optional but handy)
    "pyyaml", "pandas", "matplotlib", "tqdm",
    "tensorboard"           # optional: training logs
]:
    pip_install(pkg)
print("OK")


OK


### Notebook Sanity Check

In [5]:
import subprocess, shutil, decord, cv2, yaml, pandas as pd
print("ffmpeg:", shutil.which("ffmpeg"))
print("yt-dlp:", shutil.which("yt-dlp"))
print("decord:", decord.__version__)
print("opencv:", cv2.__version__)


ffmpeg: /usr/bin/ffmpeg
yt-dlp: None
decord: 0.6.0
opencv: 4.12.0


#### Verify environment & CUDA

In [6]:
import sys, torch, platform, subprocess, shutil
print("Python:", sys.version)
print("Executable:", sys.executable)
print("Torch:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))
print("ffmpeg:", shutil.which("ffmpeg"))
print("yt-dlp:", shutil.which("yt-dlp"))


Python: 3.11.9 (main, Jul 28 2025, 00:40:36) [GCC 14.2.0]
Executable: /home/falasoul/venvs/ai-env/bin/python
Torch: 2.7.1+cu128
CUDA available: True
GPU: NVIDIA GeForce RTX 5080
ffmpeg: /usr/bin/ffmpeg
yt-dlp: None


#### Create folders & write a “fingerprint”

In [19]:
from datetime import datetime
import json, yaml, torch, sys

# ✅ Root is already defined in the first cell
for p in ["data/raw", "data/videos", "data/metadata", "checkpoints", "runs"]:
    (root / p).mkdir(parents=True, exist_ok=True)

# Load main config
with open(root / "configs/wlasl100.yaml", "r") as f:
    C = yaml.safe_load(f)

# Record environment info
env = {
    "timestamp": datetime.utcnow().isoformat() + "Z",
    "python": sys.version,
    "executable": sys.executable,
    "torch": torch.__version__,
    "cuda_available": torch.cuda.is_available(),
}

with open(root / "env_info.json", "w") as f:
    json.dump(env, f, indent=2)

print("Environment info recorded in env_info.json")
env


Environment info recorded in env_info.json


{'timestamp': '2025-11-02T01:52:08.968920Z',
 'python': '3.11.9 (main, Jul 28 2025, 00:40:36) [GCC 14.2.0]',
 'executable': '/home/falasoul/venvs/ai-env/bin/python',
 'torch': '2.7.1+cu128',
 'cuda_available': True}

#### Optional: memory-friendly defaults

In [20]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
print("PYTORCH_CUDA_ALLOC_CONF set.")

PYTORCH_CUDA_ALLOC_CONF set.


This converts v0.3 format into the flat structure your loaders expect, and also normalizes key names (subset → split).

In [21]:
# --- run from inside notebooks/ ---
from pathlib import Path
import json

root = Path("..").resolve()
src_path = root / "data" / "metadata" / "WLASL_v0.3.json"
dst_path = root / "data" / "metadata" / "WLASL_flat.json"

with open(src_path, "r") as f:
    data = json.load(f)

flat = []
# v0.3 structure: [{ "gloss": "...", "instances": [ {...}, {...} ] }, ...]
for g in data:
    gloss = g.get("gloss")
    for inst in g.get("instances", []):
        # Normalize keys so downstream code is stable
        flat.append({
            "gloss": gloss,
            "video_id": inst.get("video_id"),
            "url": inst.get("url"),
            "split": inst.get("subset") or inst.get("split") or "train",  # normalize
            "frame_start": inst.get("frame_start"),
            "frame_end": inst.get("frame_end"),
            "fps": inst.get("fps", 25),
            "signer_id": inst.get("signer_id"),
            "bbox": inst.get("bbox"),
            "variation_id": inst.get("variation_id", 0),
            "source": inst.get("source"),
        })

with open(dst_path, "w") as f:
    json.dump(flat, f, indent=2)

print(f"Flattened {len(flat)} instances → {dst_path}")


Flattened 21083 instances → /home/falasoul/notebooks/USD/AAI-590/Capstone/AAI-590-G3-ASL/data/metadata/WLASL_flat.json


In [22]:
# Load the flat file and peek
flat = json.load(open(dst_path))
print("Total instances:", len(flat))
print("Example:", flat[0])

# Count glosses, splits
from collections import Counter
print("Unique glosses:", len(set(d["gloss"] for d in flat)))
print("Splits:", Counter(d["split"] for d in flat))


Total instances: 21083
Example: {'gloss': 'book', 'video_id': '69241', 'url': 'http://aslbricks.org/New/ASL-Videos/book.mp4', 'split': 'train', 'frame_start': 1, 'frame_end': -1, 'fps': 25, 'signer_id': 118, 'bbox': [385, 37, 885, 720], 'variation_id': 0, 'source': 'aslbrick'}
Unique glosses: 2000
Splits: Counter({'train': 14289, 'val': 3916, 'test': 2878})
